Run the notebook at TODAY after 15:00:00 (so the market is closed and data is available).

Input your current position.

The orders are for tomorrow.

In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
TODAY = str(cal.iloc[-1,0])
YESTERDAY = str(cal.iloc[-2,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-12-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-12-01
2024-12-09
2024-12-10


In [34]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 336.66
SH600570 800
SH601698 900
SH600104 1100
SH600588 1500
SH600233 1000
'''

REC_LIST = [
    ('GBDT0', 'EXP_BENCH', '33519c106eb4412794c2962dfc94af6f'), # GBDT BENCH_NOW r0 all
    ('GBDT1', 'EXP_BENCH', '9e5b313bd2fc49bf9e2d0232ba0466ed'), # GBDT BENCH_NOW r1 all
    ('GBDT2', 'EXP_BENCH', '6990a118668444a79f18067c0a658fc9'), # GBDT BENCH_A r0 all
    ('GBDT3', 'EXP_BENCH', '1dcc7b1ec14e46379690c4f2d6e5242d'), # GBDT BENCH_A r0 all
]

# EXP_NAME, rid = REC_LIST[0]
TOPK = 5
NDROP = 1
HT = 0

info = {
    'ALGO': ['GBDT'],
    'market': ['csi300_ext'], 
    # 'market': ['all'], 
    'benchmark': ["SH000300"], 
    'feat': ["Alpha360"], 
    'label': ['r0'],
    'params': [f'topk {TOPK} ndrop {NDROP} HT {HT}']
}

In [3]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2549755:MainThread](2024-12-11 10:24:40,014) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[2549755:MainThread](2024-12-11 10:24:40,021) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[2549755:MainThread](2024-12-11 10:24:40,023) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [4]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt)
print(account)
print(norm_pos)
print(hold_lst)

2024-12-10 09:00:00
{'SH600570': {'amount': 800}, 'SH601698': {'amount': 900}, 'SH600104': {'amount': 1100}, 'SH600588': {'amount': 1500}, 'SH600233': {'amount': 1000}, 'cash': 336.66}
{'_settle_type': 'None', 'position': {'SH600570': {'amount': 3727, 'price': 6.836996555328369}, 'SH601698': {'amount': 3497, 'price': 5.913832664489746}, 'SH600233': {'amount': 7140, 'price': 2.0420126914978027}, 'SH600104': {'amount': 12461, 'price': 1.586225152015686}, 'SH600588': {'amount': 7188, 'price': 2.6728668212890625}, 'cash': 336.66, 'now_account_value': 100057.30793741703}, 'init_cash': 336.66}
['SH600570', 'SH601698', 'SH600233', 'SH600104', 'SH600588']


Prepare dataset

In [5]:
nameDFilter = NameDFilter(name_rule_re='(SH60[0-9]{4})|(SZ00[0-9]{4})')
filter_pipe=[nameDFilter]
benchmark = BENCH_Step(test_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

bt_benchmark = BENCH_Step(bt_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

In [6]:
dataset = init_instance_by_config(benchmark.dataset_config)
bt_dataset = init_instance_by_config(bt_benchmark.dataset_config)
# dataset.prepare('test')

[2549755:MainThread](2024-12-11 10:25:59,905) INFO - qlib.timer - [log.py:127] - Time cost: 79.278s | Loading data Done
/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/qlib/utils/paral.py:54: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  return df.groupby(axis=axis, level=level).apply(apply_func)
[2549755:MainThread](2024-12-11 10:26:00,963) INFO - qlib.timer - [log.py:127] - Time cost: 1.055s | ProcessInf Done
[2549755:MainThread](2024-12-11 10:26:00,996) INFO - qlib.timer - [log.py:127] - Time cost: 0.032s | ZScoreNorm Done
[2549755:MainThread](2024-12-11 10:26:01,008) INFO - qlib.timer - [log.py:127] - Time cost: 0.011s | Fillna Done
[2549755:MainThread](2024-12-11 10:26:01,015) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done
[2549755:MainThread](2024-12-11 10:26:01,022) INFO - qlib.timer - [log.py:127] - Time cost: 0.005s | CSZScoreNorm Done
[2549755:MainThread](2024-12-11 10:26:0

In [7]:
label_df = bt_dataset.prepare("test")
label_df = pd.DataFrame(label_df.loc[:, 'LABEL0'])
label_df.columns = ['label']
# label_df.index = label_df.index.droplevel(0)
# label_df

Load and run model

In [8]:
def simp_eval(res_df, k):
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    top_k_ic = top_k_per_group['label'].corr(sorted_df['score'])
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win

def run_model(model):
    BT_EXP_NAME = 'backtest'

    # backtest and analysis
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, bt_dataset, recorder)
        sr.generate()
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        bt_pred_df = recorder.load_object("pred.pkl")
    res_df = pd.concat([label_df, bt_pred_df], axis=1, sort=True).reindex( label_df.index)

    top5ic, top5win = simp_eval(res_df, 5)
    EXP_NAME = 'realworld_position_maker'

    # backtest and analysis
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, dataset, recorder)
        sr.generate()
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        pred_df = recorder.load_object("pred.pkl")

    return top5ic, top5win, pred_df

In [35]:
info_dict = {}
pred_lst = []

for NAME, EXP_NAME, rid in REC_LIST:
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=rid)
        model = recorder.load_object("trained_model")
    top5ic, top5win, pred_df = run_model(model)
    info_dict[NAME] = [top5ic, top5win]
    pred_lst.append(pred_df)

[2549755:MainThread](2024-12-11 11:08:13,769) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...
[2549755:MainThread](2024-12-11 11:08:13,781) INFO - qlib.workflow - [recorder.py:341] - Recorder cbc1f22b4de44fd594eab0537d87b7df starts running under Experiment 2 ...
[2549755:MainThread](2024-12-11 11:08:13,921) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[2549755:MainThread](2024-12-11 11:08:13,927) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...
[2549755:MainThread](2024-12-11 11:08:13,935) INFO - qlib.workflow - [recorder.py:341] - Recorder 61f13906370c4220a03da75310bed1da starts running under Experiment 4 ...
[2549755:MainThread](2024-12-11 11:08:14,071) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 4
[2549755:MainThread](2024-12-11 11:08:14,082) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[2549755:M

'The following are prediction results of the LGBModel model.'
                                     score
datetime   datetime   instrument          
2024-12-02 2024-12-02 SH600000   -0.000213
                      SH600009   -0.030369
                      SH600010   -0.006537
                      SH600011   -0.001845
                      SH600015    0.019897


[2549755:MainThread](2024-12-11 11:08:14,360) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 3
[2549755:MainThread](2024-12-11 11:08:14,372) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[2549755:MainThread](2024-12-11 11:08:14,376) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[2549755:MainThread](2024-12-11 11:08:14,384) INFO - qlib.workflow - [recorder.py:341] - Recorder b24193c839a14fc8bdbd046a1a88cbdb starts running under Experiment 3 ...
[2549755:MainThread](2024-12-11 11:08:14,468) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[2549755:MainThread](2024-12-11 11:08:14,474) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...
[2549755:MainThread](2024-12-11 11:08:14,482) INFO - qlib.workflow - [recorder.py:341] - Recorder 7f51bd81adcd42438871de3232797639 starts running under Experiment 2 ...


'The following are prediction results of the LGBModel model.'
                                     score
datetime   datetime   instrument          
2024-12-10 2024-12-10 SH600000   -0.009662
                      SH600009   -0.003230
                      SH600010   -0.221178
                      SH600011   -0.026211
                      SH600015   -0.107392


[2549755:MainThread](2024-12-11 11:08:14,590) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[2549755:MainThread](2024-12-11 11:08:14,598) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...
[2549755:MainThread](2024-12-11 11:08:14,605) INFO - qlib.workflow - [recorder.py:341] - Recorder 4d40ed8c8d104f3da2bfdbeee692a955 starts running under Experiment 4 ...
[2549755:MainThread](2024-12-11 11:08:14,719) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 4
[2549755:MainThread](2024-12-11 11:08:14,726) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[2549755:MainThread](2024-12-11 11:08:14,730) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...
[2549755:MainThread](2024-12-11 11:08:14,737) INFO - qlib.workflow - [recorder.py:341] - Recorder 34bffaec11534c0f89fe70628b2d8460 starts running under Experiment 4 ...
[2549755:M

...

'The following are prediction results of the LGBModel model.'
                                     score
datetime   datetime   instrument          
2024-12-10 2024-12-10 SH600000   -0.032373
                      SH600009   -0.001847
                      SH600010   -0.126247
                      SH600011   -0.040483
                      SH600015   -0.100408


[2549755:MainThread](2024-12-11 11:08:15,805) INFO - qlib.timer - [log.py:127] - Time cost: 0.004s | waiting `async_log` Done
[2549755:MainThread](2024-12-11 11:08:15,810) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...
[2549755:MainThread](2024-12-11 11:08:15,817) INFO - qlib.workflow - [recorder.py:341] - Recorder 81112fc9b3164102bd49bfdca3e37c0c starts running under Experiment 4 ...
[2549755:MainThread](2024-12-11 11:08:15,941) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 4
[2549755:MainThread](2024-12-11 11:08:15,950) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[2549755:MainThread](2024-12-11 11:08:15,953) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...
[2549755:MainThread](2024-12-11 11:08:15,961) INFO - qlib.workflow - [recorder.py:341] - Recorder f5b4d003a15a469d950e7dd9319ab1a1 starts running under Experiment 4 ...
[2549755:M

'The following are prediction results of the LGBModel model.'
                                     score
datetime   datetime   instrument          
2024-12-02 2024-12-02 SH600000   -0.032578
                      SH600009   -0.026948
                      SH600010   -0.032967
                      SH600011    0.001782
                      SH600015    0.021102


[2549755:MainThread](2024-12-11 11:08:16,187) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 3
[2549755:MainThread](2024-12-11 11:08:16,193) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[2549755:MainThread](2024-12-11 11:08:16,196) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[2549755:MainThread](2024-12-11 11:08:16,204) INFO - qlib.workflow - [recorder.py:341] - Recorder 707d846e084e41eb951c8cc18165fb04 starts running under Experiment 3 ...
[2549755:MainThread](2024-12-11 11:08:16,286) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


'The following are prediction results of the LGBModel model.'
                                     score
datetime   datetime   instrument          
2024-12-10 2024-12-10 SH600000   -0.002996
                      SH600009   -0.007417
                      SH600010   -0.171343
                      SH600011   -0.001548
                      SH600015   -0.045808


In [36]:
info_df = pd.DataFrame(info_dict, index=['IC', 'WIN'])
report_info = info_df

In [ ]:
pred_df = pd.concat(pred_lst, axis=1)
pred_df.columns = name_lst = [r[0] for r in REC_LIST]
_pred_df = pred_df

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([pred_df[n]*info_dict[n][0] for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [38]:
rank_df = pred_df.droplevel([0,1]).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df.head(20)

In [39]:
hold_lst

['SH600570', 'SH601698', 'SH600233', 'SH600104', 'SH600588']

In [40]:
pos_rank = pred_df.droplevel([0,1]).sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
report_pos = pos_rank

In [41]:
report_info

,GBDT0,GBDT1,GBDT2,GBDT3
IC,0.481547,0.651685,0.508849,0.644207
WIN,0.566667,0.500000,0.500000,0.466667


In [42]:
report_pos

,GBDT0,GBDT1,GBDT2,GBDT3,mean,std,score
instrument,,,,,,,
SH601698,-0.109673,-0.053772,-0.165179,-0.050882,-0.094877,0.046868,-0.204685
SH600570,0.014065,-0.053357,0.000101,-0.083678,-0.030717,0.039600,-0.081854
SH600588,0.045815,-0.011633,0.030895,-0.014127,0.012737,0.026170,0.021101
SH600104,0.000543,0.039097,0.023136,0.053142,0.028980,0.019551,0.071748
SH600233,0.084382,0.087851,0.083874,0.074567,0.082668,0.004921,0.188601


In [43]:
report_top

,GBDT0,GBDT1,GBDT2,GBDT3,mean,std,score,close
instrument,,,,,,,,
SZ002129,0.168365,0.105242,0.150083,0.154850,0.144635,0.023712,0.325785,9.950001
SZ002459,0.173376,0.084794,0.159345,0.105232,0.130687,0.036735,0.287621,15.710000
SZ000963,0.163270,0.077279,0.166681,0.108591,0.128955,0.037702,0.283754,37.000004
SZ002202,0.161820,0.073516,0.145249,0.107814,0.122100,0.034198,0.269198,10.830000
SH601238,0.142715,0.060475,0.127471,0.084294,0.103739,0.032910,0.227301,9.640000
SH600438,0.174242,0.040388,0.156430,0.049291,0.105088,0.060658,0.221579,25.260000
SH601012,0.100315,0.082428,0.116572,0.092548,0.097966,0.012475,0.220961,17.620001
SH603296,0.114519,0.055714,0.117771,0.089450,0.094363,0.024860,0.209006,63.309998
SH603659,0.094840,0.061434,0.118018,0.088390,0.090671,0.020157,0.202700,18.460001
